In [1]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
!pip install xlrd=='1.1.0' gensim=='3.4.0' scikit-learn=='0.19.1' seaborn=='0.8' spaCy=='2.0.12'

# <font color=#003091> Introducción resultados Word2Vec - Simpsons </font>

- Autor original del kernel: https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial


<img src="https://images.freeimages.com/images/large-previews/2b9/letters-1-yes-1188348.jpg" alt="drawing" width="350"/>

# Briefing about Word2Vec:

<img src="http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png" alt="drawing" width="950"/>

[[1]](#References:)




In [4]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

<img src="https://upload.wikimedia.org/wikipedia/commons/4/44/Logo_The_Simpsons.svg" alt="drawing" width="400"/>

## Los datos:
Se eligió un script de los Simpson, porque tienen más de 150k lineas de diálogos y porque los Simpson es conocido

El dataset contiene los caracteres, localizaciones, detalles de episodios, y lineas de diálogos de aproximadamente 600 episodios de los Simpsons, desde 1989. El conjunto de datos se puede encontrar aquí:
https://www.kaggle.com/ambarish/fun-in-text-mining-with-simpsons/data (~25MB)

# Preprocessing

Nos quedamos solo con dos columnas:
* `raw_character_text`: la persona que habla
* `spoken_words`: Su diálogo

No se utiliza la columna `normalized_text` porque deseamos hacer nuestro propio pre-procesado.

In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/3. Embeddings/"

df = pd.read_csv(path + 'input/simpsons_dataset.csv')
df.shape

(158314, 2)

In [6]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


Los valores desconocidos provienen de la parte del diálogo donde algo sucede, pero no hay diálogo. Por ejemplo "(Springfield Elementary School: EXT. ELEMENTARY - SCHOOL PLAYGROUND - AFTERNOON)"

In [7]:
df.isnull().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

Se eliminan los valores nulos

In [8]:
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

INFO - 19:49:32: NumExpr defaulting to 2 threads.


raw_character_text    0
spoken_words          0
dtype: int64

## Limpieza
Se lematiza, se eliminan stop-words y filtra los caracteres no provenientes del alfabeto de cada diálogo

- Stemmer: https://medium.com/@tusharsri/nlp-a-quick-guide-to-stemming-60f1ca5db49e
- Lemmatizer: https://medium.com/@tusharsri/lemmatization-af85aa3e5a86

In [9]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lematiza y elimina stopwords
    # - Los doc necesitan ser objetos Doc de spacy
    txt = [token.lemma_ for token in doc if not token.is_stop]
    
    # Se filtran los diálogos con menos de 3 palabras debido a que se puede aprender poco de ellos
    if len(txt) > 2:
        return ' '.join(txt)

Filtra los caracteres no provenientes del alfabeto

In [10]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

Se utiliza la función .pipe() de SpaCy para acelerar el proceso de limpieza

In [11]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 48.63 mins


Poner los resultados en un dataframe para eliminar los valores vacíos y los duplicados

In [12]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(85964, 1)

## Bigramas
Utilizamos el paquete de Gensim Phrases para automáticamente detectar frases de dos palabras comunes (bigramas) de una lista de sentencias.
https://radimrehurek.com/gensim/models/phrases.html

La mayor razón por la que hacemos esto es para detectar palabras como "mr_burns" or "bart_simpson" !

In [13]:
from gensim.models.phrases import Phrases, Phraser

INFO - 20:38:11: 'pattern' package not found; tag filters are not available for English


Se necesita pre-procesar. `Phrases()` toma una lista de palabras como entrada 

In [14]:
sent = [row.split() for row in df_clean['clean']]

Crea bigramas interesantes

In [15]:
phrases = Phrases(sent, min_count=30, progress_per=10000)
phrases

INFO - 20:38:11: collecting all words and their counts
INFO - 20:38:11: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 20:38:11: PROGRESS: at sentence #10000, processed 63561 words and 52816 word types
INFO - 20:38:12: PROGRESS: at sentence #20000, processed 130943 words and 99866 word types
INFO - 20:38:12: PROGRESS: at sentence #30000, processed 192972 words and 138532 word types
INFO - 20:38:12: PROGRESS: at sentence #40000, processed 249842 words and 172659 word types
INFO - 20:38:12: PROGRESS: at sentence #50000, processed 311265 words and 208566 word types
INFO - 20:38:12: PROGRESS: at sentence #60000, processed 373588 words and 243702 word types
INFO - 20:38:12: PROGRESS: at sentence #70000, processed 436441 words and 278740 word types
INFO - 20:38:12: PROGRESS: at sentence #80000, processed 497829 words and 311886 word types
INFO - 20:38:13: collected 330804 word types from a corpus of 537160 words (unigram + bigrams) and 85964 sentences
INFO - 20:38:13: us

El objetivo de Phraser() es reducir el consumo de memoria de Phrases(). Mediante el descarte de los estados del modelo que no son interesantes para el aprendizaje del modelo.

In [16]:
bigram = Phraser(phrases)
bigram

INFO - 20:38:13: source_vocab length 330804
INFO - 20:38:16: Phraser built with 126 126 phrasegrams


Transformar el corpus en base a los bigramas detectados

In [17]:
sentences = bigram[sent]
sentences

## Palabras más frecuentes
Para asegurar la calidad del trabajo previo

In [18]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

30178

In [19]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['oh', 'like', 'know', 'get', 'hey', 'think', 'right', 'look', 'want', 'come']

# Entrenamiento del modelo
## Implementación Word2Vec de Gensim
Implementación Word2Vec de Gensim: https://radimrehurek.com/gensim/models/word2vec.html

In [20]:
import multiprocessing

from gensim.models import Word2Vec

## Se separa el entrenamiento en tres pasos:

1. `Word2Vec()`: 
>En este primero paso, se inicializan los parámetros del modelo uno por uno. <br>No se inicializan los parámetros `sentences`, y por ello se deja el modelo sin inicializar.
2. `.build_vocab()`: 
>En este punto se construye el vocabulario de una secuencia de frases y por tanto se inicializa el modelo.<br> Mediante los logs, se puede ver el progreso y más importante aún, el efecto de `min_count` y `sample` en el corpus de palabras. En particular los parámetros `sample` y `min_count` tienen una gran importancia en el modelo. Mostrar ambos permite un manejo más sencillo de su influencia.

3. `.train()`:
>Finalmente se entrena el modelo.<br>
Los Logs se utilizan para monitorizar que varios threads no se ejecutan simultaneamente

In [21]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

## Los parámetros:

* `min_count` <font color='purple'>=</font> <font color='green'>int</font> - Filtra todas las palabras con una frecuencia menor que la fijada - (2, 100)


* `window` <font color='purple'>=</font> <font color='green'>int</font> - Distancia máxima entre la palabra actual y la palabra a predecir - (2, 10)


* `size` <font color='purple'>=</font> <font color='green'>int</font> - Dimensionalidad del embedding - (50, 300)


* `sample` <font color='purple'>=</font> <font color='green'>float</font> - Parámetro para reducir el entrenamiento con palabras con alta frecuencia de aparición.  - (0, 1e-5)


* `alpha` <font color='purple'>=</font> <font color='green'>float</font> - Learning Rate inicial - (0.01, 0.05)


* `min_alpha` <font color='purple'>=</font> <font color='green'>float</font> - Learning rate mínimo al que se puede llegar durante el entrenamiento. Para fijar el descenso de aprendizaje: alpha - (min_alpha * epochs) ~ 0.00


* `negative` <font color='purple'>=</font> <font color='green'>int</font> - Si > 0, se va a utilizar negative sampling, el int para valores negativos especifica el número de "noise words" que se deberían utilizar. Si = 0, no se utiliza negative sampling. - (5, 20)


* `workers` <font color='purple'>=</font> <font color='green'>int</font> - Número de cores que utilizar

In [22]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

## Escritura de la Tabla Vocabulario:
Word2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them):

In [23]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 20:38:18: collecting all words and their counts
INFO - 20:38:18: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 20:38:18: PROGRESS: at sentence #10000, processed 61718 words, keeping 9558 word types
INFO - 20:38:19: PROGRESS: at sentence #20000, processed 127351 words, keeping 14506 word types
INFO - 20:38:19: PROGRESS: at sentence #30000, processed 187829 words, keeping 17619 word types
INFO - 20:38:19: PROGRESS: at sentence #40000, processed 243332 words, keeping 20385 word types
INFO - 20:38:19: PROGRESS: at sentence #50000, processed 303182 words, keeping 22878 word types
INFO - 20:38:20: PROGRESS: at sentence #60000, processed 363940 words, keeping 25200 word types
INFO - 20:38:20: PROGRESS: at sentence #70000, processed 425408 words, keeping 27401 word types
INFO - 20:38:20: PROGRESS: at sentence #80000, processed 485464 words, keeping 29275 word types
INFO - 20:38:20: collected 30178 word types from a corpus of 523700 raw words and 85964 sentence

Time to build vocab: 0.05 mins


## Entrenamiento del modelo:
_Parámetros del entrenamiento:_
* `total_examples` <font color='purple'>=</font> <font color='green'>int</font> - Contador de sentencias;
* `epochs` <font color='purple'>=</font> <font color='green'>int</font> - Número de iteracciones (epochs) sobre el corpus corpus - [10, 20, 30]

In [24]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 20:38:21: training model with 1 workers on 3319 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 20:38:22: EPOCH 1 - PROGRESS: at 31.39% examples, 63618 words/s, in_qsize 1, out_qsize 0
INFO - 20:38:23: EPOCH 1 - PROGRESS: at 67.17% examples, 65323 words/s, in_qsize 0, out_qsize 0
INFO - 20:38:24: worker thread finished; awaiting finish of 0 more threads
INFO - 20:38:24: EPOCH - 1 : training on 523700 raw words (198820 effective words) took 3.0s, 66227 effective words/s
INFO - 20:38:25: EPOCH 2 - PROGRESS: at 33.37% examples, 64205 words/s, in_qsize 0, out_qsize 0
INFO - 20:38:26: EPOCH 2 - PROGRESS: at 67.17% examples, 64438 words/s, in_qsize 0, out_qsize 0
INFO - 20:38:27: worker thread finished; awaiting finish of 0 more threads
INFO - 20:38:27: EPOCH - 2 : training on 523700 raw words (199218 effective words) took 3.0s, 66397 effective words/s
INFO - 20:38:28: EPOCH 3 - PROGRESS: at 29.42% examples, 58957 words/s, in_qsize 0, out_qsize 0


Time to train the model: 1.51 mins


Dado que no queremos entrenar el modelo más, utilizamos la función init_sims(), que hace el modelo mucho más eficiente en su uso de memoria:

In [25]:
w2v_model.init_sims(replace=True)

INFO - 20:39:52: precomputing L2-norms of word weight vectors


# Explorando el modelo
## Más similar a:

Aquí, le preguntamos al modelo que encuentre la palabra más similar a alguno de los mayores iconos de los Simpsons!

<img src="https://vignette.wikia.nocookie.net/simpsons/images/0/02/Homer_Simpson_2006.png/revision/latest?cb=20091207194310" alt="drawing" width="130"/>

Veamos lo que conseguimos con protagonista principal!

In [26]:
w2v_model.wv.most_similar(positive=["homer"])

[('bongo', 0.7772750854492188),
 ('marge', 0.7608616352081299),
 ('rude', 0.7566046118736267),
 ('wife', 0.7553014755249023),
 ('snuggle', 0.7500259876251221),
 ('sweetheart', 0.748281717300415),
 ('gee', 0.7425020933151245),
 ('crummy', 0.7420293092727661),
 ('sorry', 0.7347474098205566),
 ('worry', 0.7290295362472534)]

_Pequeña puntuación:_<br>
El conjunto de datos es la línea de diálogo de los Simpson; por lo tanto, cuando miramos las palabras más parecidas de "homer" no es necesario obtener los miembros de su familia, los rasgos de su personalidad, o incluso sus palabras más citadas. No, obtenemos lo que otros personajes (como Homer no se refiere a menudo a sí mismo en tercera persona) dijeron junto con "homer", como cómo se siente o se ve ("deprimido"), dónde está ("hamaca"), o con quién ("marge").

Veamos lo que el bigrama "homer_simpson" nos da en comparación:

In [27]:
w2v_model.wv.most_similar(positive=["homer_simpson"])

[('congratulation', 0.7782440781593323),
 ('recent', 0.7706798315048218),
 ('council', 0.7668615579605103),
 ('easily', 0.763874888420105),
 ('governor', 0.7603208422660828),
 ('hutz', 0.7457172870635986),
 ('committee', 0.7440746426582336),
 ('erotic', 0.742489218711853),
 ('simon', 0.734795331954956),
 ('viewer', 0.7345239520072937)]

<img src="https://vignette.wikia.nocookie.net/simpsons/images/0/0b/Marge_Simpson.png/revision/latest?cb=20180626055729" alt="drawing" width="150"/>

¿Qué pasa ahora con Marge?

In [28]:
w2v_model.wv.most_similar(positive=["marge"])

[('convince', 0.7752747535705566),
 ('rude', 0.7635070085525513),
 ('homer', 0.7608615756034851),
 ('ralphie', 0.7571084499359131),
 ('sorry', 0.755699098110199),
 ('becky', 0.7554717659950256),
 ('grownup', 0.7534979581832886),
 ('darling', 0.750558614730835),
 ('sure', 0.750037431716919),
 ('raccoon', 0.7488826513290405)]

<img src="https://vignette.wikia.nocookie.net/simpsons/images/6/65/Bart_Simpson.png/revision/latest?cb=20180319061933" alt="drawing" width="100"/>

Veamos a Bart ahora

In [29]:
w2v_model.wv.most_similar(positive=["bart"])

[('lisa', 0.839928925037384),
 ('homework', 0.7986600399017334),
 ('surprised', 0.7944226861000061),
 ('mom', 0.790746808052063),
 ('convince', 0.7826062440872192),
 ('upset', 0.7788929343223572),
 ('typical', 0.7725905179977417),
 ('substitute', 0.7718350887298584),
 ('hearing', 0.769727349281311),
 ('strangle', 0.7668836116790771)]

Parece que tiene sentido

<img src="https://vignette.wikia.nocookie.net/simpsons/images/9/9d/Groundskeeper_Willie.png/revision/latest?cb=20130424154035" alt="drawing" width="150"/>

Dejemos a Willie como último

## Similaridades:
Here, we will see how similar are two words to each other :

In [1]:
w2v_model.wv.similarity("moe_'s", 'tavern')

NameError: ignored

Who could forget Moe's tavern? Not Barney.

<img src="https://vignette.wikia.nocookie.net/simpsons/images/6/6c/MaggieSimpson.PNG/revision/latest?cb=20180314210204" alt="drawing" width="100"/>

In [ ]:
w2v_model.wv.similarity('maggie', 'baby')

Maggie is indeed the most renown baby in the Simpsons!

In [ ]:
w2v_model.wv.similarity('bart', 'nelson')

Bart and Nelson, though friends, are not that close, makes sense!

## Eliminar un elemento:

Aquí, lo que le pedimos al modelo es que nos quite la palabra que no pertenezca a la lista!

Entre Jimbo, Milhouse, and Kearney, quien no es un bully?

In [ ]:
w2v_model.wv.doesnt_match(['jimbo', 'milhouse', 'kearney'])

Claramente Milhouse!
<img src="https://vignette.wikia.nocookie.net/simpsons/images/9/91/Milhouse_Van_Houten_2.png/revision/latest?cb=20180429212659" alt="drawing" width="150"/>

Qué pasaría en cambio si comparamos la amistad entre Nelson, Bart, and Milhouse?

In [ ]:
w2v_model.wv.doesnt_match(["nelson", "bart", "milhouse"])

Parece que Nelson es el raro!

<img src="https://vignette.wikia.nocookie.net/simpsons/images/4/40/Picture0003.jpg/revision/latest?cb=20110623042517" alt="drawing" width="200"/>

Por último, cual es la relación entre Homer y sus dos cuñadas?

In [ ]:
w2v_model.wv.doesnt_match(['homer', 'patty', 'selma'])

Parece que claramente no es Homer!

## Diferencia por analogía:
Qué palabra es a mujer como Homer es a Marge?

In [ ]:
w2v_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

"man" viene en primera posición, parece correcto!

Qué palabra es a mujer como Bart es a hombre?

In [ ]:
w2v_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

List es la hermana de Bart, su contraparte masculino!
<img src="https://vignette.wikia.nocookie.net/simpsons/images/5/57/Lisa_Simpson2.png/revision/latest?cb=20180319000458" alt="drawing" width="100"/>

### T-SNE visualizaciones
T-SNE es un algoritmo de reducción de dimensionalidad no lineal que intenta representar datos de una mayor dimensionalidad y mantener las relaciones entre vectores<br>

Aquí tenemos un buen tutorial: https://medium.com/@luckylwk/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

Nuestro objetivo en esta sección es transformar nuestras 300 dimensiones en 2 dimensiones y descubrir patrones interesantes en los datos.<br>
Para este ejemplo se va a utilizar la implementación de scikit-learn.

Para hacer las visualizaciones más relevantes, vamos a mirar las relaciones entre la palabra buscada (en <font color='red'>**rojo**</font>), las palabras más similares del modelo en azul (en <font color="blue">**blue**</font>), y otras palabras del vocabulario en verde (en <font color='green'>**green**</font>).

In [ ]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 15 dimensions with PCA
    reduc = PCA(n_components=15).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))
    

Código inspirado por: [[2]](#References:)

## 10 Palabras más parecidas vs. 8 Palabras al azar:
Comparemos donde la representación vectorial de Homero, sus 10 palabras más similares del modelo, así como las 8 aleatorias, se encuentra en un gráfico 2D:

In [ ]:
tsnescatterplot(w2v_model, 'homer', ['dog', 'bird', 'ah', 'maude', 'bob', 'mel', 'apu', 'duff'])

Curiosamente, las 10 palabras más parecidas a Homero terminan a su alrededor, al igual que Apu y Bob, dos personajes recurrentes.

## 10 palabras más similares vs. 10 más disímiles

Esta vez, comparemos dónde se encuentra la representación vectorial de Maggie y sus 10 palabras más parecidas del modelo, con la representación vectorial de las 10 palabras más disímiles de Maggie:

In [ ]:
tsnescatterplot(w2v_model, 'maggie', [i[0] for i in w2v_model.wv.most_similar(negative=["maggie"])])

¡Genial! Maggie y sus palabras más similares forman un grupo distintivo de las palabras más disímiles, ¡es una trama realmente alentadora!

## 10 Palabras más similares vs. 11 a 20 Palabras más similares:

Por último, vamos a trazar la trama de las palabras más similares al Sr. Burns clasificado del 1 al 10 contra los clasificados del 11 al 20:

(PD: El Sr. Burns se convirtió en el Sr. Burns después del pre-procesamiento)

In [ ]:
tsnescatterplot(w2v_model, "mr_burn", [t[0] for t in w2v_model.wv.most_similar(positive=["mr_burn"], topn=20)][10:])

As we can see, and that is very nice, all the 20 words are forming one cluster around Mr. Burns.

# Pensamientos finales

Espero que hayas encontrado este tutorial útil y te hayas divertido tanto leyéndolo como yo escribiéndolo. Por favor, no duden en dejar cualquier comentario, pregunta o sugerencia que puedan tener. Nos vemos por ahí!

También, por favor, comprueben [Supportiv](http://www.supportiv.com) por ahí! (Logotipo en tamaño Simpson)

<img src="https://fontmeme.com/permalink/180904/cc3d27a8aaa88189e764ee9d02331d0d.png" alt="drawing" width="500"/>


# Materiales para una comprensión más profunda:
* Word Embeddings introduction: https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
* Word2Vec introduction: https://skymind.ai/wiki/word2vec
* Another Word2Vec introduction: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
* A great Gensim implentation tutorial: http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W467ScBjM2x
* Original articles from Mikolov et al.: https://arxiv.org/abs/1301.3781 and https://arxiv.org/abs/1310.4546


# Reconocimientos

* [Pouria Mojabi](https://www.linkedin.com/in/pouria-mojabi-1873615/), co-fouder of Supportiv Inc.

# Referencias:
* [1]. Neural Net picture: McCormick, C. (2016, April 19). Word2Vec Tutorial - The Skip-Gram Model. Retrieved from http://www.mccormickml.com
* [2]. Aneesha Bakharia Medium article: https://medium.com/@aneesha/using-tsne-to-plot-a-subset-of-similar-words-from-word2vec-bb8eeaea6229

# Ejercicios
- ¿Qué información podría sacar si entreno el W2V con listas de películas vistas por usuarios?
- ¿Podría utilizar los embeddings para detectar productos sustitutivos en supermercados? ¿Qué necesitaría? ¿Cómo se haría?
- ¿Qué sucedería si entreno el modelo con textos en distintos idiomas? ¿Podría utilizar los embeddings para determinar cuantos idiomas hay?
- OPCIONAL: 
  - Crear un vector por texto, en vez de por palabra, utilizando la función de Doc2Vec de Gensim.
  - ¿Qué resultados salen?
